<a href="https://colab.research.google.com/github/FaraazArsath/Akaike-Assignment/blob/main/nlp_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Creation of Objective Questions with Multiple Correct Answers using generative text models**

In [1]:
!pip install PyMuPDF
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.6 MB/s eta 0:00:00


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
import fitz  # PyMuPDF
import spacy
import random
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the paraphrasing model and tokenizer
paraphrase_model_name = "t5-large"
paraphrase_model = T5ForConditionalGeneration.from_pretrained(paraphrase_model_name)
paraphrase_tokenizer = T5Tokenizer.from_pretrained(paraphrase_model_name)

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")


def read_entire_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""

    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()

    return text


def tokenize_sentences(text):
    doc = nlp(text)
    return [sent.text for sent in doc.sents]


def extract_keywords(text):
    doc = nlp(text)
    keywords = [token.text for token in doc if token.is_alpha]
    return list(set(keywords))


def map_sentences_to_keywords(sentences, keywords):
    keyword_sentence_mapping = {}
    for keyword in keywords:
        keyword_sentence_mapping[keyword] = [sent for sent in sentences if keyword.lower() in sent.lower()]
    return keyword_sentence_mapping


def paraphrase_question(question):
    input_ids = paraphrase_tokenizer(question, return_tensors="pt")["input_ids"]
    paraphrased_ids = paraphrase_model.generate(input_ids, num_beams=4, num_return_sequences=1)
    paraphrased_question = paraphrase_tokenizer.decode(paraphrased_ids[0], skip_special_tokens=True)
    return paraphrased_question


def get_mca_questions(context: str):
    sentences = tokenize_sentences(context)
    keywords = extract_keywords(context)
    keyword_sentence_mapping = map_sentences_to_keywords(sentences, keywords)
    mca_questions = []

    for i in range(3):
        keyword = random.choice(keywords)
        sentence = random.choice(keyword_sentence_mapping[keyword])

        # Paraphrase the sentence
        paraphrased_sentence = paraphrase_question(sentence)

        words = [token.text for token in nlp(paraphrased_sentence) if token.is_alpha]
        options = random.sample(words, 4)  # Randomly select 4 words for options

        correct_options = random.sample(["(a)", "(b)", "(c)", "(d)"], 2)

        options = random.sample(options, 4)  # Randomly shuffle options
        options = [option.capitalize() for option in options]  # Capitalize options
        formatted_question = f"Q{i+1}: {paraphrased_sentence.strip()}?\n"  # Add a question mark
        formatted_question += f"(a) {options[0]}\n(b) {options[1]}\n(c) {options[2]}\n(d) {options[3]}\n"
        formatted_question += f"Correct Options: {' & '.join(correct_options)}"  # Indicate two correct options
        mca_questions.append(formatted_question)

    return mca_questions


pdf_path = "chapter-2.pdf"
context = read_entire_pdf(pdf_path)  # Define context here
mca_questions = get_mca_questions(context)

# Print the generated questions with two lines between each set
for i, question in enumerate(mca_questions, start=1):
    print(question)
    if i < len(mca_questions):
        print("\n")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Q1: he was himself cross-examined by the British Parliament in 1772.?
(a) Cross
(b) The
(c) British
(d) Himself
Correct Options: (a) & (b)


Q2: . His office – the Collectorate – became the new centre of power and?
(a) Power
(b) Became
(c) Of
(d) Centre
Correct Options: (d) & (c)


Q3: . The fine qualities of cotton and silk produced in India had a big market in?
(a) Qualities
(b) In
(c) Cotton
(d) Produced
Correct Options: (c) & (b)
